In [ ]:
"""
To work with alpaca you will need to make an account and get your key and secret. I have mine in a separate file that is gitignored and not pushed. 
Eventually we should figure out how to automate this refresh process as as of now I am doing it manually.

API key and secret can be found in the "home" page of your alpaca account when you are logged in on the right side of the screen after scrolling a bit

Having wscat installed will also be very useful for testing straight from the command line

Additionally I have a virtual environment folder set up in this moby folder that has its own list of requirements. We can see if all packages are compatible 
and have one top level virtual environment later on but for now this was just done for the sake of simplicity
"""

In [1]:
"""
Websocket streaming doc: https://docs.alpaca.markets/docs/streaming-market-data

Basic auth command with wscat: {"action": "auth", "key": "{KEY_ID}", "secret": "{SECRET}"

If authentication is successful you will get the message: [{"T":"success","msg":"authenticated"}]

Once you are authenticated, you can subscribe to a give symbol (ticker)

A subscription is essentially what allows us to listen for data on a given symbol and constantly stream data for it. 
With the free version we can listen to a maximum of 30 symbols at once.

To subscribe to one or more symbols you can use the following command after connecting:

{
  "action": "subscribe",
  "<channel1>": ["<SYMBOL1>"],
  "<channel2>": ["<SYMBOL2>","<SYMBOL3>"],
  "<channel3>": ["*"]
}

Example:

"""


'\nWebsocket streaming doc: https://docs.alpaca.markets/docs/streaming-market-data\n\nBasic auth command with wscat: {"action": "auth", "key": "{KEY_ID}", "secret": "{SECRET}"\n\nIf authentication is successful you will get the message: [{"T":"success","msg":"authenticated"}]\n\nOnce you are authenticated, you can subscribe to a give symbol (ticker)\n\nA subscription is essentially what allows us to listen for data on a given symbol and constantly stream data for it. \nWith the free version we can listen to a maximum of 30 symbols at once.\n\nTo subscribe to one or more symbols you can use the following command after connecting:\n\n{\n  "action": "subscribe",\n  "<channel1>": ["<SYMBOL1>"],\n  "<channel2>": ["<SYMBOL2>","<SYMBOL3>"],\n  "<channel3>": ["*"]\n}\n\nExample:\n\n'

Schemas and Data formatting info:

Stocks: https://docs.alpaca.markets/docs/real-time-stock-pricing-data

Options: https://docs.alpaca.markets/docs/real-time-option-data

News: https://docs.alpaca.markets/docs/real-time-stock-pricing-data

Crypto: https://docs.alpaca.markets/docs/real-time-crypto-pricing-data

learn dat black-scholes mrthod bro





In [1]:
from api_keys import api_key, api_secret
import numpy as np
import alpaca
import json
import websockets
import asyncio

In [2]:
#The version of the streaming function below does not work in a jupyter notebook as it already runs an event loop. the version in the next cell has been slightly modified to work around this
#however when developing the project it will correspond to the version in this cell instead

alpaca_stream_url = "wss://stream.data.alpaca.markets/v2/iex"

async def stream_market_data(symbols_trades=None, symbols_quotes=None, symbols_bars=None):
    """
    each of the three parameters define which symbols we are getting the specified data for

    trades are just that, specific trades that have been placed on that given security 
    in the time frame

    quotes are the real-time latest price at which a security can be bought or sold.
    a quote consists of two prices, the bid (highes price a buyer will pay) and the ask
    (lowest price a seller will accept).

    Bars represent the 


    
    """
    async with websockets.connect(alpaca_stream_url) as ws:
        print("Openend connection")
        print(api_key)
        print(api_secret)
        #authenticate with api key and secret
        auth_message  ={
            "action" : "auth", 
            "key_id" : api_key,
            "secret_key" : api_secret
        }


        await ws.send(json.dumps(auth_message))

        #build the subscription message
        subscribe_message = {"action" : "subscribe"}
        if symbols_trades:
            subscribe_message["trades"] = symbols_trades
        if symbols_quotes:
            subscribe_message["quotes"] = symbols_quotes
        if symbols_bars:
            subscribe_message["bars"] = symbols_bars
        

        await ws.send(json.dumps(subscribe_message))

        print(f"Subscribed: {subscribe_message}")
        

        async for message in ws:
            data = json.loads(message)
            print(data)

symbols_trades = ["AAPL", "MSFT"]
symbols_quotes = ["AAPL", "MSFT"]
symbols_bars = ["AAPL"] 

asyncio.run(stream_market_data(symbols_trades, symbols_quotes, symbols_bars))

RuntimeError: asyncio.run() cannot be called from a running event loop

## Schema Returned by the trades streaming:

param name : data type, param description
T: string, message type

S: string, symbol/ticker

i: int, trade ID

x: string, exchange code where trade occurred

p: "number" (this is what it says on the website, not sure what it means but I will look into it), the trade price

s: int array<string> trade condition (list below)

t: string, timestamp (nanosecond precision pretty cool stuff!)

z: string, tape, refers to a consolidated market data stream that aggregates quotes and trades from multiple exchanges. In this context it is just the market data feed that we are sourcing data from

### Trade condition symbol list:


@ - Regular Sale
A standard, on-exchange trade occurring during normal market conditions.

I - Odd Lot Trade
A trade involving fewer than 100 shares. Odd lots do not always update the NBBO.

F - Intermarket Sweep Order (ISO)
The trade resulted from an ISO, meaning liquidity was intentionally swept across venues.

T - Extended Hours Trade
The trade occurred outside regular trading hours (pre-market or after-hours).

O - Opening Trade
A trade associated with the market opening process.

C - Closing Trade
A trade associated with the market closing process.

H - Trading Halt
A trade reported in the context of a halted security (typically late or regulatory).

L - Sold Last (Late Report)
A trade reported late to the tape.

Z - Sold (Out of Sequence)
A trade that occurred earlier but was reported out of sequence.

4 - Derivatively Priced Trade
A trade priced relative to another instrument (e.g., spread or contingent trade).

9 - Corrected Trade
A correction to a previously reported trade.

## Schema Returned by quotes streaming

T: string, message type

S: string, symbol/ticker

ax: string, ask exchange code (1-3 digit alphanumeric code that is used to identify specific market or exchange where a security's lowest asking price is listed)

ap: number, ask price

as: int, ask size in round lots 
a lot is just a fixed number of financial assets traded on an exchange, for stocks a round lot is typically defined as 100 shares or some multiple of 100 shares

bx: string, bid exchange code, denotes the exchange upon which the bid is placed

bp: number, bid price

c: array<string>, quote condition. A quote condition is a code assigned by an exchange to a specific bid/ask price quote to describe the state of the market for that security
at that precise moment. It indicates whether the quoted price is a regular, actionable market rate, or if it is subject to special circumstances. See quote condition mapping at the end of the cell

t: string, timestamp

z: string, tape (see definition in previous cell)



### Exchange Codes Mapping:
A – AMEX - NYSE MKT, LLC (NYSE MKT)<br>
B – BOST - NASDAQ OMXBX Inc.(NASDAQ OMX BX)<br>
C – CINC - National Stock Exchange, Inc. (NSX)<br>
D – NAQS - Financial Industry Regulatory Authority, Inc. (FINRA)<br>  
E – SYS - Market Independent (Generated by SIP)<br>
I – ISLD - International Securities Exchange, LLC (ISE)<br>
J – DEA - Bats EDGA Exchange, Inc. (Bats EDGA)<br>
K – DEB - Bats EDGX Exchange, Inc. (Bats EDGX)<br>
M – CHIC - Chicago Stock Exchange, Inc. (CHX)<br>
N – NYSE - New York Stock Exchange, LLC (NYSE)<br>
P – PAC - NYSE Arca, Inc. (NYSE Arca)<br>
Q – QNMS - NASDAQ OMX S – SYS - Consolidated Tape System (CTS)<br> 
T – LCQS - NASDAQ Stock Market, LLC (NASDAQ)<br> 
V – SYS - Investors’ Exchange, LLC. (IEX)<br>
W – CBOE - CBOE Stock Exchange, Inc. (CBSX)<br>
X – PHIL - NASDAQ OMX PSX, Inc. (NASDAQ OMX PSX)<br> 
Y – BATSY - Bats BYX Exchange, Inc. (Bats BYX)<br> 
Z – BATS - Bats BZX Exchange, Inc. (Bats BZX)<br>


### Quote Conditions:
R - Regular Quote: 
A standard, regular, firm quote with no special conditions. This is the most common case.

A - Slow Quote (Ask Side):
 Indicates the ask side of the quote is considered slow relative to the current market state 
(often due to liquidity replenishment or exchange conditions).

B - Slow Quote (Bid Side):
Indicates the bid side of the quote is considered slow.

E / F - Slow Quote (Both Sides / Market Conditions):
Indicates broader slow-quote conditions, typically exchange-driven (exact meaning depends on feed and tape).

N - Non-Firm Quote:
The quote is not firm; market participants may not be obligated to execute at this price.

H - Trading Halt:
The quote was disseminated while the security was halted.

O - Opening Quote:
Quote associated with the opening auction or opening market state.

C - Closing Quote:
Quote associated with the closing auction or closing market state.
It is important to consider quote conditions 

Important:
The codes are defined by the data feed and can differ between IEX and SIP

Importance of Quotes overall:

Liquidity Assessment: narrow bid-ask spread indicated higher liquidity, while lower spread indicates lower liquidity and higher transaction costs

Order Routing: Traders use these conditions to determine where to route orders for the best execution

Market sentiment: monitoring changes in quote size and frequency help gauge buying or selling pressure, which enables traders to identify trades or potential reversals

## Schema Returned by Bars streaming


Bars are essentially trade aggregates. They display a security's open, high, low, and close prices and can be used with differing granularity to see how the price of a security
fluctuated in a given time period. See the following link for instructions on how to read bars.

https://www.dummies.com/article/business-careers-money/personal-finance/investing/investment-vehicles/stocks/technical-analysis-how-to-read-a-basic-price-bar-157600/

NOTE: this article gives an example for each bar representing a whole day, but like I mentioned before the granularity can be different and we will probably be using bars that have minute long intervals represented


Attribute	Type	Description
T -	string - message type: “b”, “d” or “u”<br>
S - string	- symbol<br>
o - number	- open price<br>
h - number	- high price<br>
l - number - low price<br>
c - number - close price<br>
v - int - volume (number of trades placed on interval)<br>
vw - number - volume-weighted average price. This metric represents the average price a security trades at throughout the interval. Calculated by dividing the total dollar value of all transactions (price * volume) by the total volume. If the stock price is above the VWAP, market sentiment is generally bullish; below indicates bearish. Institutional investors use it to minimize market impact by trading near the average, while traders use it to spot entry/exit points (e.g., buying on dips below VWAP). (worth doing more research into usage) <br>
n - int	- number of trades<br>
t - string - RFC-3339 formatted timestamp<br>

In [4]:

import json
import websockets

alpaca_stream_url = "wss://stream.data.alpaca.markets/v2/iex"

async def stream_market_data_notebook(symbols_trades=None, symbols_quotes=None, symbols_bars=None):
    print("Connecting to Alpaca market data stream...")
    async with websockets.connect(alpaca_stream_url) as ws:
        
        #authenticate
        auth_message = {
            "action": "auth",
            "key":api_key,
            "secret": api_secret
        }
        await ws.send(json.dumps(auth_message))
        print("Sent auth message.")

        #build subscription message
        subscribe_msg = {"action": "subscribe"}
        if symbols_trades:
            subscribe_msg["trades"] = symbols_trades
        if symbols_quotes:
            subscribe_msg["quotes"] = symbols_quotes
        if symbols_bars:
            subscribe_msg["bars"] = symbols_bars

        await ws.send(json.dumps(subscribe_msg))
        print("Subscribed with:", subscribe_msg)

        # receive messages
        async for message in ws:
            data = json.loads(message)
            print(data)

#example symbols to track
symbols_trades = ["AAPL", "MSFT"]
symbols_quotes = ["AAPL", "MSFT"]
symbols_bars = ["AAPL"]  # minute bars if needed

# jupyter-safe execution
await stream_market_data_notebook(symbols_trades, symbols_quotes, symbols_bars)


Connecting to Alpaca market data stream...
Sent auth message.
Subscribed with: {'action': 'subscribe', 'trades': ['AAPL', 'MSFT'], 'quotes': ['AAPL', 'MSFT'], 'bars': ['AAPL']}
[{'T': 'success', 'msg': 'connected'}]
[{'T': 'success', 'msg': 'authenticated'}]
[{'T': 'subscription', 'trades': ['AAPL', 'MSFT'], 'quotes': ['AAPL', 'MSFT'], 'bars': ['AAPL'], 'corrections': ['AAPL', 'MSFT'], 'cancelErrors': ['AAPL', 'MSFT']}]
[{'T': 'q', 'S': 'AAPL', 'bx': 'V', 'bp': 272.14, 'bs': 200, 'ax': 'V', 'ap': 276, 'as': 100, 'c': ['R'], 'z': 'C', 't': '2026-02-09T18:04:27.159624599Z'}]
[{'T': 't', 'S': 'MSFT', 'i': 17880, 'x': 'V', 'p': 412.33, 's': 9, 'c': ['@', 'I'], 'z': 'C', 't': '2026-02-09T18:04:27.176749591Z'}]
[{'T': 'q', 'S': 'MSFT', 'bx': 'V', 'bp': 412.31, 'bs': 40, 'ax': 'V', 'ap': 412.36, 'as': 40, 'c': ['R'], 'z': 'C', 't': '2026-02-09T18:04:27.266302802Z'}, {'T': 'q', 'S': 'MSFT', 'bx': 'V', 'bp': 412.31, 'bs': 40, 'ax': 'V', 'ap': 414.4, 'as': 40, 'c': ['R'], 'z': 'C', 't': '2026-02

CancelledError: 

Openend connection
Subscribed: {'action': 'subscribe', 'trades': ['AAPL', 'MSFT'], 'quotes': ['AAPL', 'MSFT'], 'bars': ['AAPL']}
[{'T': 'success', 'msg': 'connected'}]
[{'T': 'error', 'code': 402, 'msg': 'auth failed'}]
[{'T': 'error', 'code': 401, 'msg': 'not authenticated'}]


ConnectionClosedError: no close frame received or sent

In [ ]:
# wss://stream.data.alpaca.markets/{version}/{feed}
#Version refers to the actual version of the service 
#feed refers to the actual place from which the information will be sourced
#see the following link for more information https://docs.alpaca.markets/docs/historical-stock-data-1#feed-parameter
#for the purposed of this demo we will use v2/iex as it is ideal for initial app testing and situations where precise pricing may not be the primary focus.
# It's a single US exchange that accounts for approximately ~2.5% of the market volume.




In [ ]:
import yfinance as yf

def detect_large(symbol : str , trade_data : dict, bar_data : dict):
    """
    Used to detect large single transactions for a stock on the user's watchlist
    Different thresholds for what is considered large based on the market cap gotten from yahoo finance

    trade_data: a json formatted as a python dictionary. In  this function we simply use the 

    """

    #trade data from alpaca
    purchase_price = trade_data["p"] 
    trade_size = trade_data["s"]

    #calculate percentage of rolling 5 minute volume
    rolling_volume = sum(bar["v"] for bar in bar_data)

    if rolling_volume > 0:
        ratio = trade_size/rolling_volume
    else:
        return None
    

    
    

    